<a href="https://colab.research.google.com/github/Nunez350/DataScienceAndAnalytics/blob/main/Module8%20/CollaborativeFiltering/softcompute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the standard papackages
!pip install numpy
!pip install pandas
!pip install fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 1.7 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29881 sha256=19bed00d3eaa68fed3321e86a9da328065d44dcb182b813250fae8b432ecb7ef
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11330 sha256=3c76ec03645e5265feb8b79e4f01ae4a2ab3df364ab48b17097090ac78925ea7
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [ ]:
# Impute necessary packages
import numpy as np
import pandas as pd
from fancyimpute import BiScaler
from soft_impute import SoftImpute
from functionsCF import GenerateTrainingSet

In [ ]:
# Read jester data from files- point to where data is stored, small set of jester dataset
# 100836 (rows), userId	jokeId	rating	timestamp (columns)
# read in values only
# name the variable "rating"
# YOUR CODE HERE
# file_path = 'jester-data-3.csv'
rating = pd.read_csv('ratings.csv', sep=',').values
# raise NotImplementedError()

In [ ]:
# Show the top 5 rows
print(rating[:5, :])

[[1.00000000e+00 1.00000000e+00 4.00000000e+00 9.64982703e+08]
 [1.00000000e+00 3.00000000e+00 4.00000000e+00 9.64981247e+08]
 [1.00000000e+00 6.00000000e+00 4.00000000e+00 9.64982224e+08]
 [1.00000000e+00 4.70000000e+01 5.00000000e+00 9.64983815e+08]
 [1.00000000e+00 5.00000000e+01 5.00000000e+00 9.64982931e+08]]


In [ ]:
# First, create an empty matrix
matrix_incomplete = np.zeros((len(np.unique(rating[:,0])), len(np.unique(rating[:,1]))))

In [ ]:
# Second, Since some movies don't have any ratings, we only use the movies that have ratings.
# Here we correspondingly change the movie IDs to make each column has ratings.
# create an array of all movie IDs
usedID = np.unique(rating[:, 1])

In [ ]:
# replace the movie IDs by the their positions in the array we just created
for i in range(len(rating[:,1])):
    rating[:,1][i] = np.where(usedID==rating[:,1][i])[0][0] + 1

In [ ]:
# all components are nan by default
matrix_incomplete[:] = np.nan
# create the index pair of the components with ratings
indices = np.array(rating[:,0] - 1).astype(int), np.array(rating[:,1] - 1).astype(int)
# change the values in the corresponding positions to the known rating information
matrix_incomplete[indices] = rating[:,2]

In [ ]:
train_indices, validation_indices = GenerateTrainingSet(rating[:,0], rating[:,1], 0.70)

In [ ]:
matrix_train = matrix_incomplete.copy()
matrix_train[:] = np.nan
matrix_train[train_indices] = matrix_incomplete[train_indices]

In [ ]:
# Create the BiScaler model
biscaler = BiScaler(scale_rows=False, scale_columns=False, max_iters=50, verbose=False)

In [ ]:
# Rescale both rows and columns to have zero mean
matrix_train_normalized = biscaler.fit_transform(matrix_train)

In [ ]:
# Use softImpute to complete the matrix. J means the number of archetypes and rand_seed means the
# seed for the inner random number generator, verbose control whether outputting algorithm logs.
softImpute = SoftImpute(J = 4, maxit = 200, random_seed = 1, verbose = False)

In [ ]:
matrix_train_softImpute = softImpute.fit(matrix_train_normalized)
# Use the softImpute model to create the predicted matrix. If we set copyto as True, then it
# directly change the value of matrix_train_normalized
matrix_train_filled_normalized = matrix_train_softImpute.predict(matrix_train_normalized, copyto = False)
# Inverse transformation to undo the scaling
matrix_train_filled = biscaler.inverse_transform(matrix_train_filled_normalized)